In [1]:
!pip install --user seaborn

You are using pip version 10.0.0, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [6]:
from sklearn.utils import shuffle
from utils.data import prepare_data
from utils.gini import eval_gini, gini_xgb
import numpy as np
import pandas as pd

# Laboratoire 2

Comme vu en classe, la classification est une part importante de la science des données. Elle consiste à apprendre à classer des échantillons et de par la suite prédire les classes de nouveaux échantillons. 

Durant ce laboratoire, vous allez travailler sur un concours Kaggle dont le but est de prédire la probabilité qu'un conducteur fasse une réclamation d'assurance auto durant l'année suivante. En voyant ce problème, la tâche semble être celle d'une regression et non d'un classification, cependant, avec les données disponibles, c'est effectivement une classification. En effet, les cibles ne sont pas des réels mais bien des entiers naturels puisque l'assureur peut simplement noter si le client a oui ou non fait une réclamation durant l'année d'après. Mais je vous conseille d'aller sur https://www.kaggle.com/c/porto-seguro-safe-driver-prediction pour avoir plus d'information.

Vous l'aurez donc compris, il faut entraîner un prédicteur probabilistique pour avoir des probabilités durant la prédiction (ex: Regression Logistique, Bayes Naif, etc. et non SVM).

Votre première tâche sera donc d'entraîner un classifieur sur l'ensemble d'entraînement et de prédire des probabilités sur l'ensemble de test.

### Preparation des données

Pour ce laboratoiree le but est surtout d'étudier la classification. La préparation des données est donc faite pour vous. 

In [48]:
data_path='./input'

train = pd.read_csv(data_path+'/train.csv')
test = pd.read_csv(data_path+'/test.csv')

prep = prepare_data(train,test)
train, targets, test = prep(True,False)

0 duplicates dropped.

Under-sampling...
Rate to undersample records with target=0: 0.34043569687437886
Number of records with target=0 after undersampling: 195246

Checking missing values...
Variable ps_ind_02_cat has 103 records (0.05%) with missing values
Variable ps_ind_04_cat has 51 records (0.02%) with missing values
Variable ps_ind_05_cat has 2256 records (1.04%) with missing values
Variable ps_reg_03 has 38580 records (17.78%) with missing values
Variable ps_car_01_cat has 62 records (0.03%) with missing values
Variable ps_car_02_cat has 2 records (0.00%) with missing values
Variable ps_car_03_cat has 148367 records (68.39%) with missing values
Variable ps_car_05_cat has 96026 records (44.26%) with missing values
Variable ps_car_07_cat has 4431 records (2.04%) with missing values
Variable ps_car_09_cat has 230 records (0.11%) with missing values
Variable ps_car_11 has 1 records (0.00%) with missing values
Variable ps_car_14 has 15726 records (7.25%) with missing values
In total

## Classifieur classique

**QUESTION 1:** Entraînez un classifieur classique sur l'ensemble X_train et retournez le score Gini pour cet ensemble, puis prédisez les probabilités sur l'ensemble de test. Pour avoir les meilleurs résultats possibles (score gini), faites une recherche d'hyper-paramètres (ex:GridSearchCV) et **montrez votre travail**.

In [49]:
X, y = train.as_matrix()[:,1:], targets.as_matrix()
X, y = shuffle(X,y)
cutoff = int(len(X)*0.9)

X_train_valid, y_train_valid = X[:cutoff], y[:cutoff]
X_test, y_test = X[cutoff:], y[cutoff:]
X_sub = test.as_matrix()[:,1:]

del X, y, train, targets, test

<font color=blue>**REP**: La classifieur choisi est la regression logistique. Utilisons un GridSearch pour trouver les meilleurs parametres:

In [45]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from time import time, clock

clf = Pipeline([
        ('clf', LogisticRegression())
    ]);

'''
        Parameters
        ----------
        penalty : string, 'l1' or 'l2' (default='l2')
            Used to specify the norm used in the penalization. The 'newton-cg',
            'sag' and 'lbfgs' solvers support only l2 penalties.
        dual : bool, (default=True)
            Dual or primal formulation. Dual formulation is only implemented for
            l2 penalty with liblinear solver. Prefer dual=False when
            n_samples > n_features.
        tol : float, optional (default=1e-4)
            Tolerance for stopping criteria.
        C : float, optional (default=1.0)
            Inverse of regularization strength; must be a positive float.
            Like in support vector machines, smaller values specify stronger
            regularization.
        fit_intercept : boolean, optional (default=True)
            Specifies if a constant (a.k.a. bias or intercept) should be
            added to the decision function.
        max_iter : int, (default=1000)
            The maximum number of iterations to be run.
        solver : string, {'newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'}, (default: 'liblinear')
            Algorithm to use in the optimization problem.
            - For small datasets, 'liblinear' is a good choice, whereas 'sag' and
                'saga' are faster for large ones.
            - For multiclass problems, only 'newton-cg', 'sag', 'saga' and 'lbfgs'
                handle multinomial loss; 'liblinear' is limited to one-versus-rest
                schemes.
            - 'newton-cg', 'lbfgs' and 'sag' only handle L2 penalty, whereas
                'liblinear' and 'saga' handle L1 penalty.
            Note that 'sag' and 'saga' fast convergence is only guaranteed on
            features with approximately the same scale. You can
            preprocess the data with a scaler from sklearn.preprocessing.
'''

parameters = {
        'clf__penalty':('l1','l2'),
        'clf__tol':(1e-2, 1e-4,),
        'clf__C': (0.1, 1.0),
        'clf__fit_intercept': (True, False),
        'clf__max_iter':(300, 100,),
        'clf__solver':('liblinear','saga')
    }

grid_search = GridSearchCV(clf, parameters,  verbose=1, iid=True, cv=4, n_jobs=-1, return_train_score=False)

print("Performing grid search...")
print("parameters:")
print(parameters)
t0 = time()
    
grid_search.fit(X_train_valid,y_train_valid)
print("done in %0.3fs" % (time() - t0))
print()
    
# Meilleurs parametres
best_set = grid_search.best_params_
print("Meilleurs parametres trouves : ", best_set)


best_estimator = grid_search.best_estimator_
score_train = grid_search.best_score_
print("Best score for validation set : %0.3f" % score_train)

    
probs = best_estimator.predict_proba(X_test)[:,1]

# Calculer le score Gini
score_test = eval_gini(y_test,probs)
print("Best score for test set : %0.3f" % score_test)

Performing grid search...
parameters:
{'clf__penalty': ('l1', 'l2'), 'clf__tol': (0.01, 0.0001), 'clf__C': (0.1, 1.0), 'clf__fit_intercept': (True, False), 'clf__max_iter': (300, 100), 'clf__solver': ('liblinear', 'saga')}
Fitting 4 folds for each of 64 candidates, totalling 256 fits


/usagers/gizag/.local/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usagers/gizag/.local/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usagers/gizag/.local/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usagers/gizag/.local/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usagers/gizag/.local/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reac

/usagers/gizag/.local/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed: 16.8min
/usagers/gizag/.local/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usagers/gizag/.local/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usagers/gizag/.local/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usagers/gizag/.local/lib/python3.6/site-packages/sklearn/line

done in 1464.315s

Meilleurs parametres trouves :  {'clf__C': 1.0, 'clf__fit_intercept': False, 'clf__max_iter': 300, 'clf__penalty': 'l1', 'clf__solver': 'saga', 'clf__tol': 0.01}
Best score for validation set : 0.900
Best score for test set : 0.273


<font color=blue>**REP:** Les Meilleurs parametres trouves sont :  {'C': 1.0, 'fit_intercept': False, 'max_iter': 300, 'penalty': 'l1', 'solver': 'saga', 'tol': 0.01} qui donne un score Gini test de **0.285** </font> 

In [56]:
# Initialiser votre classifieur
clf = LogisticRegression(C=1.0, fit_intercept=False, max_iter=300, penalty='l1', solver='saga', tol=0.01)

# Entraîner le classifieur
clf.fit(X_train_valid,y_train_valid)

# Calculer les probabilités prédites sur l'ensemble de test
probs = clf.predict_proba(X_test)[:,1]

# Calculer le score Gini
score_test = eval_gini(y_test,probs)
print("Best score for test set : %0.3f" % score_test)

Best score for test set : 0.285


## Méthodes d'ensembles

Les méthodes d'ensembles sont très populaires dans le machine learning pour donner de très bons résultats rapidement avec des classifieurs simples. Dans la communauté Kaggle, particulièrement, ces méthodes sont souvent utilisées et ne nécessitent pas des capacités de computation très élevées contraîrement aux réseaux de neurones.

Le principe général est simple: combiner plusieurs classifieurs simples afin d'obtenir un classifieur qui généralise mieux. 

Pour rappel, l'erreur de généralisation peut se décomposer en deux termes: 


*   Le biais: si la famille de fonction considérée ne contient pas la fonction idéale
*   La variance: variabilité dans la focntion trouvée due à la variabilité de l'ensemble de données

La deuxième partie du laboratoire consite donc à se familiariser avec ces méthodes et de déterminer si elles aident pour cette tâche.

### Boosting

Le boosting a principalement pour but de réduire le biais. Sont principe est le suivant:


1.   Entraîner un classifieur simple $h_1$ sur l'ensemble de donnée initial et prédire les classes.
2.   Déterminer les échantillons mal classés et entraîner un autre classifieur $h_{t+1}$ en augmentant l'importance de ces échantillons dans le calcul de la perte.
3.   Combiner tous les classifieurs avec des poids différents ($\alpha_t$) pour chaque classifieurs pour obtenir les prédictions.

  Pour une prédiction binaire {-1,1}:      $H(x) = sign(\sum_{t=1}^T \alpha_t h_t(x))$

4.   Répéter étapes 2, 3 N fois.



**QUESTION 2**: Reprenez le classifieur précédent en rajoutant du boosting et comparez les résultats. 

HINT: Extreme Gradient Boosting semble fonctionner particulièrement bien pour ce problème.

<font color=blue> **REP:** Utilisons pour cela la fonction **XGBClassifier()** pour ameliorer le classifieur precedemment identifie. Le resultat Score gini est ainsi ameliore a **0.295** </font> 

In [60]:
def BoostedClassifier(X_train_, y_train_, X_valid_, y_valid_):
    
    from sklearn.datasets import make_hastie_10_2
    import xgboost as xgb
    from xgboost.sklearn import XGBClassifier
    
    # Initialiser le classifieur Boosté
    clf = XGBClassifier(objective = "reg:logistic", n_estimators=100, learning_rate=0.1,max_depth=5, random_state=0)


    # Entraîner le classifieur avec la métrique gini_xgb (ci-poosible)
    clf.fit(X_train, y_train, xgb_model=None)
    
    # Calculer les probabilités prédites par le meilleur estimateur sur l'ensemble de validation
    valid_probs =  clf.predict_proba(X_valid)[:,1]
    
    # Calculer le score Gini pour les probabilités valid_probs
    score_valid = eval_gini(y_valid_,valid_probs)
    print("Best score for valid set : %0.3f" % score_valid)

    # Calculer les probabilités prédites par le meilleur estimateur sur l'ensemble de test
    test_probs = clf.predict_proba(X_test)[:,1]
    
    return test_probs, score_valid

In [61]:
# Séparer X_train_valid, y_train_valid en X_train, X_valid et y_train, y_valid avec un split de 0.9 (90% des données dans train et 10% dans test)
cutoff = int(len(X_train_valid)*0.9)
X_train, y_train = X_train_valid[:cutoff], y_train_valid[:cutoff]
X_valid, y_valid = X_train_valid[cutoff:], y_train_valid[cutoff:]

# Appeler le BoostedClassifier
test_probs, score_valid = BoostedClassifier(X_train, y_train, X_valid, y_valid)

# Calculer le score Gini pour les probabilités test_probs
score_test = eval_gini(y_test,test_probs)
print("Best score for test set : %0.3f" % score_test)

Best score for valid set : 0.274
Best score for test set : 0.295


### Boosting + Bagging

La bagging est une méthode relativement simple qui a pour but de réduire la variance. Le principe est le suivant:
1.   Séparer l'ensemble d'entraînement en N ensembles.
2.   Entraîner N classifieurs simples sur les N ensembles.
3.   Produire N sets de prédictions sur l'ensemble de validation avec ces prédicteurs 
4.   Faire la moyenne de ces prédictions

      $H(x) = \dfrac{1}{N} \sum_{t=1}^N h_t(x)$

**QUESTION 3**: Utiliser la méthode du bagging en réutilisant l'algorithme boosté précédant. BONUS si vous obtenez plus de 0.3 de score Gini sur l'ensemble de test.

<font color=blue>**REP**: Utilisons pour cela la fonction **BalancedBaggingClassifier()** du module imblearn qui permet en plus du Bagging, d'avoir un bagging ou les proprtions des echantillons sont balances pour etre similaires aux proportions observees dans les donnees initiales. 

In [63]:
# Faire le Bagging de votre clasifieur bossté.
from sklearn.ensemble import BaggingClassifier
from sklearn.datasets import make_hastie_10_2
from sklearn.ensemble import GradientBoostingClassifier
from imblearn.ensemble import BalancedBaggingClassifier 
import xgboost as xgb
from xgboost.sklearn import XGBClassifier

# Initialiser le classifieur Boosté et avec Bagging Balance
clf = BalancedBaggingClassifier(XGBClassifier(objective = "reg:logistic", n_estimators=100, learning_rate=0.1,max_depth=5, random_state=0))

# Faire la moyenne des probabilitées sur l'ensemble de test
clf.fit(X_train_valid, y_train_valid) 
test_probs = clf.predict_proba(X_test)[:,1]

# Calculer le score Gini sur l'ensemble de test
score_test = eval_gini(y_test,test_probs)
print("Balanced -- Best score for test set : %0.3f" % score_test)

Balanced -- Best score for test set : 0.297


<font color=blue>**REP**: Le Score Gini est ainsi ameliore a **0.297**! avec le bagging balance et le boosting.
Notons que nous pourrions ameliorer davantage les resultats en faisant un **GridSearch** pour obtenir les meilleurs parametres avec les fonction de boosting et de bagging.</font> 

# Pour soumettre les résultats

In [ ]:
import csv

with open("submission.csv", "w", newline="") as csvfile:
    csv_writer = csv.writer(csvfile, delimiter=',', quoting=csv.QUOTE_NONE)
    csv_writer.writerow(["id","target"])
    for i, prob in enumerate(test_probs):
        csv_writer.writerow([i,prob])

Excecutez la commande:

`kaggle competitions submit -c porto-seguro-safe-driver-prediction -f submission.csv -m "Message"`